In [1]:
# Import libraries
import numpy as np
import xarray as xr
import pandas as pd

In [88]:
# Specify filepaths and filenames.
filepath = '/home/jovyan/ML_MB_Norway_data/'
filename = '2023-06-01_stake_mb_Hydra2_corrected.csv'
filename_cleaned = '2023-06-01_stake_mb_norway_cleaned.csv'

In [77]:
# Load data.
data = pd.read_csv(filepath + filename, sep=';')

# Rename columns.
data = data.rename(columns={"utm_east3": "utm_east_approx", 
                            "utm_north4": "utm_north_approx", 
                            "altitude5": "altitude_approx"})

data

,glacier_id,utm_zone,utm_east_approx,utm_north_approx,altitude_approx,location_description,location_id,stake_no,altitude,utm_east,utm_north,balance_winter,balance_summer,balance_netto,dt_prev_year_min_date,dt_curr_year_max_date,dt_curr_year_min_date,stake_remark,flag_correction
0,54,34,529400.0,7780050.0,500.0,Nederst på bretunga,L10,L1012,471.0,529480.0,7780100.0,1.32,-6.24,-4.92,25.09.2012 00:00,14.05.2013 00:00,07.11.2013 00:00,NaN,netto
1,54,34,529400.0,7780050.0,500.0,Nederst på bretunga,L10,L1013,439.0,529453.0,7780088.0,1.68,-4.35,-2.67,07.11.2013 00:00,09.05.2014 00:00,24.09.2014 00:00,NaN,NaN
2,54,34,529400.0,7780050.0,500.0,Nederst på bretunga,L10,L1013,444.0,529438.0,7780082.0,0.76,-4.17,-3.41,24.09.2014 00:00,08.05.2015 00:00,23.09.2015 00:00,NaN,NaN
3,54,34,529400.0,7780050.0,500.0,Nederst på bretunga,L10,L1013,444.0,529438.0,7780082.0,0.78,-4.37,-3.59,23.09.2015 00:00,23.05.2016 00:00,22.09.2016 00:00,Snødyp ut fra stakemålingene (0.65 m) virker i...,netto
4,54,34,529400.0,7780050.0,500.0,Nederst på bretunga,L10,L1016,468.0,529348.0,7780059.0,1.59,-4.05,-2.46,22.09.2016 00:00,25.04.2017 00:00,29.09.2017 00:00,Sondert snødyp.,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4196,3141,32,351800.0,6647500.0,1080.0,Nederst på bretunga,B15,B1509,1078.0,351790.0,6647480.0,1.50,-3.57,-2.07,01.10.2012 00:00,03.05.2012 00:00,12.10.2012 00:00,Kombinasjon med stake 15-12,NaN
4197,3141,32,351800.0,6647500.0,1080.0,Nederst på bretunga,B15,B1509,1078.0,351787.0,6647465.0,1.14,-4.29,-3.15,12.10.2012 00:00,29.05.2013 00:00,24.09.2013 00:00,Endring av stakelengden fra 7. august til 24. ...,NaN
4198,3141,32,351800.0,6647500.0,1080.0,Nederst på bretunga,B15,B1514,1079.0,351773.0,6647494.0,1.19,-3.80,-2.61,14.10.2014 00:00,22.04.2015 00:00,14.10.2015 00:00,"Stakemålt snødyp var 3.0 m, mens to sonderinge...",NaN
4199,3141,32,351800.0,6647500.0,1080.0,Nederst på bretunga,B15,B1514,1078.0,351785.0,6647480.0,1.72,-4.38,-2.66,14.10.2015 00:00,28.04.2016 00:00,06.10.2016 00:00,Basert på sondert snødyp.,NaN


In [78]:
# New columns indicating if location/altitude is approximate. Fill new column with "N" for
# location/altitude is not approximate.
data['approx_loc'] = 'N'
data['approx_altitude'] = 'N'

# If "utm_east" values are missing, fill column "approx_loc" with "Y" indicating
# that the location is approximate.
data.loc[data['utm_east'].isna(), 'approx_loc'] = 'Y'

# If "altitude" values are missing, fill column "approx_altitude" with "Y"
# indicating that the altitude is approximate.
data.loc[data['altitude'].isna(), 'approx_altitude'] = 'Y'

# Where there is no exact location, fill inn approximate location based on
# "utm_east_approx" and "utm_north_approx" in columns "utm_east" and "utm_north".
# Location is now filled for every observation, with column "approx_loc" indicating
# wether location is approximate (Y) or exact (N).
approx_locs_east = data.loc[data['approx_loc'] == 'Y', 'utm_east_approx']
data.loc[data['approx_loc'] == 'Y', 'utm_east'] = approx_locs_east
approx_locs_north = data.loc[data['approx_loc'] == 'Y', 'utm_north_approx']
data.loc[data['approx_loc'] == 'Y', 'utm_north'] = approx_locs_north

# Same operation with missing altitude, fill in values from "altitude_approx"
# in column "altitude". 
approx_alt = data.loc[data['approx_altitude'] == 'Y', 'altitude_approx']
data.loc[data['approx_altitude'] == 'Y', 'altitude'] = approx_alt 

# Calculate difference between approximate and exact positions and altitude as
# a measure of precision/quality of approximate locations.
data['diff_north'] = data['utm_north'] - data['utm_north_approx']
data['diff_east'] = data['utm_east'] - data['utm_east_approx']
data['diff_altitude'] = data['altitude'] - data['altitude_approx']

In [79]:
# View data
data

,glacier_id,utm_zone,utm_east_approx,utm_north_approx,altitude_approx,location_description,location_id,stake_no,altitude,utm_east,...,dt_prev_year_min_date,dt_curr_year_max_date,dt_curr_year_min_date,stake_remark,flag_correction,approx_loc,approx_altitude,diff_north,diff_east,diff_altitude
0,54,34,529400.0,7780050.0,500.0,Nederst på bretunga,L10,L1012,471.0,529480.0,...,25.09.2012 00:00,14.05.2013 00:00,07.11.2013 00:00,NaN,netto,N,N,50.0,80.0,-29.0
1,54,34,529400.0,7780050.0,500.0,Nederst på bretunga,L10,L1013,439.0,529453.0,...,07.11.2013 00:00,09.05.2014 00:00,24.09.2014 00:00,NaN,NaN,N,N,38.0,53.0,-61.0
2,54,34,529400.0,7780050.0,500.0,Nederst på bretunga,L10,L1013,444.0,529438.0,...,24.09.2014 00:00,08.05.2015 00:00,23.09.2015 00:00,NaN,NaN,N,N,32.0,38.0,-56.0
3,54,34,529400.0,7780050.0,500.0,Nederst på bretunga,L10,L1013,444.0,529438.0,...,23.09.2015 00:00,23.05.2016 00:00,22.09.2016 00:00,Snødyp ut fra stakemålingene (0.65 m) virker i...,netto,N,N,32.0,38.0,-56.0
4,54,34,529400.0,7780050.0,500.0,Nederst på bretunga,L10,L1016,468.0,529348.0,...,22.09.2016 00:00,25.04.2017 00:00,29.09.2017 00:00,Sondert snødyp.,NaN,N,N,9.0,-52.0,-32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4196,3141,32,351800.0,6647500.0,1080.0,Nederst på bretunga,B15,B1509,1078.0,351790.0,...,01.10.2012 00:00,03.05.2012 00:00,12.10.2012 00:00,Kombinasjon med stake 15-12,NaN,N,N,-20.0,-10.0,-2.0
4197,3141,32,351800.0,6647500.0,1080.0,Nederst på bretunga,B15,B1509,1078.0,351787.0,...,12.10.2012 00:00,29.05.2013 00:00,24.09.2013 00:00,Endring av stakelengden fra 7. august til 24. ...,NaN,N,N,-35.0,-13.0,-2.0
4198,3141,32,351800.0,6647500.0,1080.0,Nederst på bretunga,B15,B1514,1079.0,351773.0,...,14.10.2014 00:00,22.04.2015 00:00,14.10.2015 00:00,"Stakemålt snødyp var 3.0 m, mens to sonderinge...",NaN,N,N,-6.0,-27.0,-1.0
4199,3141,32,351800.0,6647500.0,1080.0,Nederst på bretunga,B15,B1514,1078.0,351785.0,...,14.10.2015 00:00,28.04.2016 00:00,06.10.2016 00:00,Basert på sondert snødyp.,NaN,N,N,-20.0,-15.0,-2.0


In [80]:
# 4194 of 4201 points. A total of 7 rows are missing both altitude and altitude_approx.
data_crop_alt = data[data['diff_altitude'].notna()]

# 4053 of 4201 points. A total of 148 rows are missing both exact loc and approx loc.
data_crop_loc = data[data['diff_east'].notna()]

# Cleaned dataset with 4046 instances. A total of 155 points are either missing 
# both exact and approximate coordinates or altitude.
data_crop = data_crop_alt[data_crop_alt['diff_east'].notna()]

data_crop.reset_index(inplace=True)

# Check balances:
data_crop['diff_netto'] = data_crop['balance_netto'] - (data_crop['balance_winter'] + data_crop['balance_summer'])


/tmp/ipykernel_195/2921847822.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_crop['diff_netto'] = data_crop['balance_netto'] - (data_crop['balance_winter'] + data_crop['balance_summer'])


In [81]:
data_crop

,index,glacier_id,utm_zone,utm_east_approx,utm_north_approx,altitude_approx,location_description,location_id,stake_no,altitude,...,dt_curr_year_max_date,dt_curr_year_min_date,stake_remark,flag_correction,approx_loc,approx_altitude,diff_north,diff_east,diff_altitude,diff_netto
0,0,54,34,529400.0,7780050.0,500.0,Nederst på bretunga,L10,L1012,471.0,...,14.05.2013 00:00,07.11.2013 00:00,NaN,netto,N,N,50.0,80.0,-29.0,0.000000e+00
1,1,54,34,529400.0,7780050.0,500.0,Nederst på bretunga,L10,L1013,439.0,...,09.05.2014 00:00,24.09.2014 00:00,NaN,NaN,N,N,38.0,53.0,-61.0,0.000000e+00
2,2,54,34,529400.0,7780050.0,500.0,Nederst på bretunga,L10,L1013,444.0,...,08.05.2015 00:00,23.09.2015 00:00,NaN,NaN,N,N,32.0,38.0,-56.0,0.000000e+00
3,3,54,34,529400.0,7780050.0,500.0,Nederst på bretunga,L10,L1013,444.0,...,23.05.2016 00:00,22.09.2016 00:00,Snødyp ut fra stakemålingene (0.65 m) virker i...,netto,N,N,32.0,38.0,-56.0,0.000000e+00
4,4,54,34,529400.0,7780050.0,500.0,Nederst på bretunga,L10,L1016,468.0,...,25.04.2017 00:00,29.09.2017 00:00,Sondert snødyp.,NaN,N,N,9.0,-52.0,-32.0,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4041,4196,3141,32,351800.0,6647500.0,1080.0,Nederst på bretunga,B15,B1509,1078.0,...,03.05.2012 00:00,12.10.2012 00:00,Kombinasjon med stake 15-12,NaN,N,N,-20.0,-10.0,-2.0,0.000000e+00
4042,4197,3141,32,351800.0,6647500.0,1080.0,Nederst på bretunga,B15,B1509,1078.0,...,29.05.2013 00:00,24.09.2013 00:00,Endring av stakelengden fra 7. august til 24. ...,NaN,N,N,-35.0,-13.0,-2.0,4.440892e-16
4043,4198,3141,32,351800.0,6647500.0,1080.0,Nederst på bretunga,B15,B1514,1079.0,...,22.04.2015 00:00,14.10.2015 00:00,"Stakemålt snødyp var 3.0 m, mens to sonderinge...",NaN,N,N,-6.0,-27.0,-1.0,0.000000e+00
4044,4199,3141,32,351800.0,6647500.0,1080.0,Nederst på bretunga,B15,B1514,1078.0,...,28.04.2016 00:00,06.10.2016 00:00,Basert på sondert snødyp.,NaN,N,N,-20.0,-15.0,-2.0,0.000000e+00


In [82]:
# Remove rows for Nigardsbreen in 1982 where stake locations are inaccurate.

# Set stake_no to string type.
data_crop["stake_no"]= data_crop["stake_no"].astype('string')

#remove1 = data_crop.loc[(data_crop['stake_no']=='N5082') & (data_crop['balance_summer']==-6.32)]
idx1 = data_crop[(data_crop['stake_no']=='N5082') & (data_crop['balance_summer']==-6.32)].index
data_crop.drop(idx1, inplace=True)

#remove2 = data_crop.loc[(data_crop['stake_no']=='N5182') & (data_crop['balance_summer']==-6.26)]
idx2 = data_crop[(data_crop['stake_no']=='N5182') & (data_crop['balance_summer']==-6.26)].index
data_crop.drop(idx2, inplace=True)

data_crop

/tmp/ipykernel_195/1797613825.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_crop["stake_no"]= data_crop["stake_no"].astype('string')
/tmp/ipykernel_195/1797613825.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_crop.drop(idx1, inplace=True)
/tmp/ipykernel_195/1797613825.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_crop.drop(idx2, inplace=True)


,index,glacier_id,utm_zone,utm_east_approx,utm_north_approx,altitude_approx,location_description,location_id,stake_no,altitude,...,dt_curr_year_max_date,dt_curr_year_min_date,stake_remark,flag_correction,approx_loc,approx_altitude,diff_north,diff_east,diff_altitude,diff_netto
0,0,54,34,529400.0,7780050.0,500.0,Nederst på bretunga,L10,L1012,471.0,...,14.05.2013 00:00,07.11.2013 00:00,NaN,netto,N,N,50.0,80.0,-29.0,0.000000e+00
1,1,54,34,529400.0,7780050.0,500.0,Nederst på bretunga,L10,L1013,439.0,...,09.05.2014 00:00,24.09.2014 00:00,NaN,NaN,N,N,38.0,53.0,-61.0,0.000000e+00
2,2,54,34,529400.0,7780050.0,500.0,Nederst på bretunga,L10,L1013,444.0,...,08.05.2015 00:00,23.09.2015 00:00,NaN,NaN,N,N,32.0,38.0,-56.0,0.000000e+00
3,3,54,34,529400.0,7780050.0,500.0,Nederst på bretunga,L10,L1013,444.0,...,23.05.2016 00:00,22.09.2016 00:00,Snødyp ut fra stakemålingene (0.65 m) virker i...,netto,N,N,32.0,38.0,-56.0,0.000000e+00
4,4,54,34,529400.0,7780050.0,500.0,Nederst på bretunga,L10,L1016,468.0,...,25.04.2017 00:00,29.09.2017 00:00,Sondert snødyp.,NaN,N,N,9.0,-52.0,-32.0,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4041,4196,3141,32,351800.0,6647500.0,1080.0,Nederst på bretunga,B15,B1509,1078.0,...,03.05.2012 00:00,12.10.2012 00:00,Kombinasjon med stake 15-12,NaN,N,N,-20.0,-10.0,-2.0,0.000000e+00
4042,4197,3141,32,351800.0,6647500.0,1080.0,Nederst på bretunga,B15,B1509,1078.0,...,29.05.2013 00:00,24.09.2013 00:00,Endring av stakelengden fra 7. august til 24. ...,NaN,N,N,-35.0,-13.0,-2.0,4.440892e-16
4043,4198,3141,32,351800.0,6647500.0,1080.0,Nederst på bretunga,B15,B1514,1079.0,...,22.04.2015 00:00,14.10.2015 00:00,"Stakemålt snødyp var 3.0 m, mens to sonderinge...",NaN,N,N,-6.0,-27.0,-1.0,0.000000e+00
4044,4199,3141,32,351800.0,6647500.0,1080.0,Nederst på bretunga,B15,B1514,1078.0,...,28.04.2016 00:00,06.10.2016 00:00,Basert på sondert snødyp.,NaN,N,N,-20.0,-15.0,-2.0,0.000000e+00


In [83]:
#data_crop['balance_netto'].isna().sum()
#data_crop['diff_netto'].isna().sum()
#data_crop['balance_annual'] = data_crop['balance_netto'] + data_crop['diff_netto']

In [89]:
# Save cleaned dataset
data_crop.to_csv(filepath + filename_cleaned, index=False)  